# Indexing, Selection, and Operations

Maintenant que nous connaissons bien les structures de données des pandas, nous pouvons nous intéresser à certaines des caractéristiques intermédiaires des cadres de données, notamment
    
- le découpage, l'indexation et les sous-ensembles de grands ensembles de données sur la base d'une étiquette intelligente
- **L'étiquetage hiérarchique** des axes
- **Le tri et le classement** des données dans les DataFrames
- La manipulation aisée des **données manquantes**.
- Des outils de **summarisation** des données


Dans cette section, nous allons manipuler les données recueillies auprès des navires de haute mer sur la côte est des Etats Unis. Les opérations des navires sont surveillées à l'aide du **Automatic Identification System (AIS)**, une technologie de navigation de sécurité en mer que les navires sont tenus d'entretenir et qui utilise des transpondeurs pour transmettre des signaux radio à très haute fréquence (VHF) contenant des informations statiques, notamment le nom du navire, l'indicatif d'appel et le pays d'origine, ainsi que des informations dynamiques propres à un voyage particulier, telles que la position, le cap et la vitesse du navire. 

![AIS](images/ais.gif)

La Convention internationale pour la sauvegarde de la vie humaine en mer de l'Organisation Maritime Internationale (OMI) exige des capacités AIS fonctionnelles sur tous les navires de plus de 300 tonnes et les garde-côtes américains exigent l'AIS sur presque tous les navires naviguant dans les eaux américaines. Les gardes-côtes ont mis en place un réseau national de récepteurs AIS qui couvre la quasi-totalité des eaux américaines. **Les signaux AIS** sont transmis plusieurs fois par minute et le réseau est capable de traiter des milliers de rapports par minute et des mises à jour aussi souvent que toutes les deux secondes. Par conséquent, un voyage typique dans notre étude peut inclure la transmission de centaines ou de milliers de signaux codés AIS. Cela fournit une riche source de données spatiales qui comprend des informations à la fois **spatiales et temporelles**.

Pour nos besoins, nous utiliserons des **données résumées** qui décrivent le transit d'un navire donné dans une zone administrative particulière. Les données comprennent l'heure de début et de fin du segment de transit, ainsi que des informations sur la vitesse du navire, la distance parcourue, etc.

In [ ]:
import pandas as pd
import numpy as np

vessels = pd.read_csv('./data/AIS/vessel_information.csv', index_col=0)

## Indexing et Selection

L'indexation est similaire à celle qu'on utilise dans des arrays, à l'exception près qu'on peut extraire les objets `Index`, en plus des index entiers.

In [ ]:
# Exemple de Series index
flag = vessels.flag
flag

In [ ]:
# Numpy-style indexing
flag[:10]

In [ ]:
# Indexing par label
flag[[298716,725011300]]

Dans un `DataFrame` nous pouvons faire des sélections sur les deux axes :

In [ ]:
vessels[['num_names','num_types']].head()

In [ ]:
vessels[vessels.max_loa>700]

L'attribut d'index `loc` permet de choisir des sous-ensembles de manière intuitive :

In [ ]:
vessels.loc[720768000, ['names','flag', 'type']]

In [ ]:
vessels.loc[:4731, 'names']

La sélection peut se faire aussi avec des labels texte:

In [ ]:
vessels.loc[:310, 'flag':'loa']

En plus de l'attribut `loc` qui permet de faire une séléction par **label**, pandas permet aussi la séléction par **position** avec l'attribut `iloc`.

Ainsi, nous pouvons demander une position absolue dans le tableau : 

In [ ]:
vessels.iloc[:5, 5:8]

### Exercice

Vous pouvez utiliser la méthode `isin` pour faire une requête dans le DataFrame selon une liste de valeurs : 

    data['color'].isin(['red', 'blue'])

Utiliser `isin` pour trouver tous les navires enregistrés au Danemark ou au Japon. Combien d'entrées nous trovons ?

In [ ]:
# Write your answer here

## Indexing avec `where`

Les `DataFrame` Pandas  ont aussi un appel de type `where` qui accepte des conditions. Les objets retournés gardent l'index du `DataFrame` original, qui ne change pas de forme. Ceci est très important pour préserver l' **allignment** lors des opérations entre `DataFrame`s.

In [ ]:
np.random.seed(42)
normal_vals = pd.DataFrame({'x{}'.format(i):np.random.randn(100) for i in range(5)})

normal_vals.head()

In [ ]:
normal_vals.where(normal_vals>0).head()

`where` inclut une option `other` qui accepte une valeur scalaire ou tabulaire pour remplacer la valeur de `DataFrame` si la condition n'est pas satisfaite.

Par exemple, nous pouvons retourner la valeur absolue de X comme ici :

In [ ]:
normal_vals.where(normal_vals>0, other=-normal_vals).head()

Il est aussi possible de faire appel à une fonction `lambda` afin de modifier la valeur :

In [ ]:
normal_vals.where(normal_vals>0, other=lambda y: -y*100).head()

L'opération `mask` est l'inverse booléenne de `where` :

In [ ]:
normal_vals.mask(normal_vals>0).head()

## Sélection avec  `query`

Parfois, la sélection avec les index peut demander une commande trop élaborée car il faut associer plusieurs appels au Dataframe.

In [ ]:
normal_vals[(normal_vals.x1 > normal_vals.x0) & (normal_vals.x3 > normal_vals.x2)].head()

Pour une syntaxe plus lisible et concise, nous pouvons utiliser la méthode `query`. Au lieu d'entrer toute la spécification de la colonne, nous pouvons simplement passer une string. Voici le même exemple réécrit avec query :

In [ ]:
normal_vals.query('(x1 > x0) & (x3 > x2)').head()

Les objets `DataFrame.index` et `DataFrame.columns` sont inclus par défaut dans les requêtes query. Is nous souhaitons passer une variable, nous pouvons utiliser le préfixe `@`:

In [ ]:
min_loa = 700

In [ ]:
vessels.query('max_loa > @min_loa')

## Opérations

Les objets `DataFrame` et `Series` supportent plusieurs opérations au sein d'un objet ou entre différents objets.

Dans cette section nous allons charger un dataset supplémentaire avec des informations sur le déplacements de chaque navire. Ce dataset est bien plus important car chaque navire a fait plusieurs voyages au long des années.

Par exemple, nous pouvons faire des opérations arithmetiquessur des éléments des deux objets, en combinant par exemple les distances voyagées au long des années. Commençons d'abord à créer des Series pour regrouper les distances parcouroues en 2009 et 2010 :

In [ ]:
segments = pd.read_csv('./data/AIS/transit_segments.csv', parse_dates=['st_time'])

Ensuite, on extrait l'année (on verra plus sur les données temporelles d'ici peu)

In [ ]:
segments['year'] = segments.st_time.dt.year
segments.year.head()

In [ ]:
segments2009 = segments[segments.year==2009].copy()
lengths2009 = pd.Series({name: segments2009[segments2009.name==name].seg_length.sum() 
                         for name in segments2009.name.unique()})

In [ ]:
segments2010 = segments[segments.year==2010].copy()
lengths2010 = pd.Series({name: segments2010[segments2010.name==name].seg_length.sum() 
                         for name in segments2010.name.unique()})

Autre option plus efficace est d'utiliser groupby, qu'on verra plus tard.

In [ ]:
length2009 = segments.loc[segments.year==2009, 'seg_length']
length2009.index = segments.name[segments.year==2009]

length2010 = segments.loc[segments.year==2010, 'seg_length']
length2010.index = segments.name[segments.year==2010]

Maintenant, on peut faire la somme des distances parcourues :

In [ ]:
seg_lengths = lengths2009 + lengths2010
seg_lengths

L'alignement des données de Pandas inclut des valeurs `NaN` pour les labels qui ne sont pas communs aux deux Series. En effet, on voit que la plupart des navires n'a que des données pour une seule année.

In [ ]:
seg_lengths.notnull().mean()

Bien que cela correspond à la réalité, on ne veut pas de valeurs `NaN` dans notre tableau. nous pouvons remplacer les valeurs `NaN` lors de la somme en indiquant d'utiliser zéro à la place d'un `NaN` :

In [ ]:
lengths2009.add(lengths2010, fill_value=0)

Les opérations peuvent aussi être diffusées (**broadcast**) entre les lignes et colonnes.

Par exemple, si on soustrait la valeur maximum de LOA (longueur du navire) de la colonne `max_loa` column, nous avons la différence de taille entre les navires et le plus long des bateaux (en pieds, bien sûr).

In [ ]:
vessels.max_loa - vessels.max_loa.max()

Nous pouvons aussi comparer des élements  ligne par ligne :

In [ ]:
vessels[vessels.max_loa==vessels.max_loa.max()]

In [ ]:
recs = vessels[['num_names','num_loas', 'max_loa', 'num_types']]
diff = recs - recs.loc[354092000]
diff[:10]

Finalement, nous pouvons aussi appliquer des fonctions aux lignes ou colonnes d'un `DataFrame`

In [ ]:
recs.apply(np.median)

In [ ]:
def range_calc(x):
    return x.max() - x.min()

In [ ]:
recs.apply(range_calc)

## Tri et Classement

Les objets Pandas incluent des méthodes pour réordonner les données.

In [ ]:
segments.sort_index().head()

In [ ]:
segments.sort_index(ascending=False).head()

Essayez d'ordonner les **colonnes** en ordre croissant, au lieu des lignes :

In [ ]:
segments.sort_index(axis=1).head()

Il est aussi possible d'utiliser `sort_values` pour trier un `Series`par valeur au lieu du label.

In [ ]:
segments.seg_length.sort_values(ascending=False).head(10)

Pour un `DataFrame`, nous pouvons trier selon les valeurs de plusieurs colonnes en passant un argument `by` à  `sort_values` :

In [ ]:
segments[['avg_sog','max_sog','min_sog']].sort_values(ascending=[False,True], 
                                           by=['max_sog', 'min_sog']).head(10)

## Indexation Hiérarchique

Le champ `mmsi` (Maritime Mobile Service Identity) représent un identificateur dans le tablea `vessels`, mais pas dans le tableau `segments`.

Nous pouvons utiliser l'indexation hiérarchique afin de créer un **index unique** avec plus d'information que l'index entier donné par défaut.

In [ ]:
segments_hier = segments.set_index(['mmsi', 'name', 'year', 'transit', 'segment']).sort_index(ascending=True)
segments_hier.head(10)

Cet index est un objet `MultiIndex`, composé d'une séquence de tuples (dans ce cas, la composition de plusieurs colonnes). Lorsqu'on trouve de valeurs répétés, pandas ne les affiche pas, rendant plus simple l'identification des groupes de valeurs.

In [ ]:
segments_hier.index.is_unique

Essayez d'utiliser cet index hiérarchique pour récupérer le premier segment du 10ème trajet effectué par le navire *Sentinel* (mmsi=366766980) :

In [ ]:
segments_hier.loc[(366766980, 'Sentinel', 2009, 10, 1)]

Avec un index hiérachique, nous pouvons sélectionner des sous-ensemble des données à partir d'un index *partiel*  :

In [ ]:
segments.loc[9]

Les indices hiérarchiques peuvent aussi être crées sur d'autres axes (lignes). Voici un exemple :

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape(( 4, 3)), 
                  index =[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], 
                  columns =[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

frame

Pour rendre le traitement plus simple, on peut renommer les index avec ces valeurs :

In [ ]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

Et à partir de cela, faire tout type de recherche : 

In [ ]:
frame.loc['a', 'Ohio']

#### Exercice :Récupérer la valeur de `b2` dans `Colorado` :

In [ ]:
# votre réponse ici

Additionally, the order of the set of indices in a hierarchical `MultiIndex` can be changed by swapping them pairwise:

L'une des façons la plus simple d'extraire des données partielles avec un `MultiIndex` est d'utiliser la fonction `IndexSlice`. Par exemple, si nous voulons les deux premiers segments de toutes les voyages en 2009 et 2010 :

In [ ]:
segments_hier.loc[pd.IndexSlice[:, :, 2009:2010, [1,2]], 'seg_length'].head(12)

Le `:` dit à pandas d'inclure tout le niveau (0 à n-1).

Les données peuvent aussi être ordonnées selon un niveau, avec la fonction `sort_index` et l'argument `level`  spécifié :

In [ ]:
segments.sort_index(level='max_sog', ascending=False).head()

## Données Manquantes

Il est très probable que vous datasets contiennent de données manquantes. Pandas offre plusieurs outils pour les traiter et ainsi vous simplifier la préparation des données.

Les données manquantes sont représentées par des `NaN`. Toutefois, les `None` sont aussi considérées comme des données manquantes car d'autres bibliothèques les utilisent ainsi (ex : NumPy).

In [ ]:
foo = pd.Series([np.nan, -3, None, 'foobar'])
foo

In [ ]:
foo.isnull()

Pour illustrer le traitement de données manquantes avec Pandas, vous allez importer un dataset qui regroupe des résultats de tests faits sur des enfants avec des difficultés auditives. Parmi les variables (colonnes), plusieurs ne sont pas renseignées ;

In [ ]:
testing = pd.read_csv('./data/test_scores.csv', index_col=0)
testing.head(10)

Les lignes (ou colonnes) avec de données manquantes peuvent tout simplement être supprimées, si vous avec encore assez pour continuer à travailler. 

Par défaut, `dropna` supprime toute ligne où l'une des valeurs est manquante.

In [ ]:
testing.dropna().head(10)

Un résultat similaire peut être obtenu avec une recherche

In [ ]:
testing.isnull().head()

In [ ]:
testing[testing.notnull()].head()

Ce comportement "efface si une seule colonne est NaN" peut être allégé avec l'option `how='all'`, qui ne supprime la ligne que si toutes les colonnes sont vides.

In [ ]:
testing.dropna(how='all')

Il est possible de régler ce seuil avec l'argument `thresh`.

In [ ]:
testing.dropna(thresh=10)

### Exercice

En utilisant l'argument `axis`, essayez de supprimer les `colonnes` avec des champs manquants :

In [ ]:
# Votre réponse ici

Parfois il est plus intéressant de remplir les cases manquants, plutôt que de supprimer les lignes/colonnes. La valeur de remplissage peut être une valeur par défaut (0, par exemple), ou une valeur calculée à partir de l'ensemble des données valides (la moyenne, par exemple). La fonction `fillna` sert à ça.

In [ ]:
testing.fillna(0).head(10)

In [ ]:
testing.fillna({'family_inv': 0, 'mother_hs':1}).head(10)

Observez que `fillna` retourne un nouveau DataFrame, au lieu de modifier l'objet existant (**en général, c'est une bonne pratique car on n'efface pas les sources !**).

Les valeurs manquantes peuvent aussi être interpolées :

In [ ]:
testing.fillna(method='bfill').head(10)

## Agrégation de Données

Souvent on veut agréger les données dans les `Series` ou `DataFrame` pour mieux les comprendre ou comparer. Voici quelques exemples d'opérations.

In [ ]:
testing.sum()

In [ ]:
testing.mean()

Des fois on ne veut pas ignorer les `nan`.

In [ ]:
testing.mean(skipna=False)

Si on passe l'argument `axis=1`, l'opération sera appliquée aux lignes au lieu des colonnes, ce qui peut être utile dans certaines situations.

In [ ]:
nonenglish_nonwhite_withHS = testing[['non_english', 'mother_hs', 'non_white']].prod(axis=1)

nonenglish_nonwhite_withHS

La fonction `describe` regroupe plusieurs de ces aggrégations, ce qui lui rend très pratique pour un aperçu des données :

In [ ]:
testing.describe()

`describe` peut détecter des données non-numeriques et rendre des informations utiles.

In [ ]:
testing.non_english.describe()

Nous pouvons aussi effecuter des vérifications rapides telles que la covariance ou la correlation entre deux variables : 

$$cov(x,y) = \sum_i (x_i - \bar{x})(y_i - \bar{y})$$

In [ ]:
testing.score.cov(testing.age_test)

$$corr(x,y) = \frac{cov(x,y)}{(n-1)s_x s_y} = \frac{\sum_i (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_i (x_i - \bar{x})^2 \sum_i (y_i - \bar{y})^2}}$$

In [ ]:
testing.score.corr(testing.age_test)

Exercice : Effectuer les test `corr` sur tout le DataFrame `testing` :

In [ ]:
# Votre réponse ici

## Exporter les DataFrames sur des fichiers

Une fois les données traitées et nettoyés, vous voulez certainement les sauvegarder. Plusieurs options sont disponibles. La plus simples est d'utiliser le format csv :

In [ ]:
testing.to_csv("testing.csv")

Cette fonction `to_csv` exporte un `DataFrame` vers un fichier séparé par des virgules (comma-separated values  - csv). Vous pouvez spécifier des délimiteurs spécifiques (avec l'argument `sep`), comment les valeurs manquantes seront représentées (avec l'argument `na_rep`), si l'index doit être écrit aussi (l'argument `index` ), si l'entête (noms des colonnes) doit être écrit (`header`), etc.

Un fichier csv reste toutefois très volumeux, plusieurs options de stockage binaires sont disponibles. Le format sérialisé de Python (pickle) est une option.

In [ ]:
testing.to_pickle("testing_pickle")

l'opposé de `to_pickle` est la fonction `read_pickle`, qui recrée un `DataFrame` ou `Series`:

In [ ]:
pd.read_pickle("testing_pickle").head()

## References

[Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do) Wes McKinney